# Análise Exploratória e Inferencial de Dados de E-commerce

**Equipe:** Levi Moraes e Thays Barbosa

Este notebook contém a análise exploratória de dados (EDA) e a análise inferencial (Intervalos de Confiança) para o e-commerce brasileiro, conforme solicitado pela direção. O objetivo é fornecer respostas confiáveis sobre receita, margens, frete, prazos de entrega e comportamento do cliente.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.proportion import proportion_confint
import statsmodels.stats.api as sms
import os

# Configuração do diretório de dados
data_dir = "../../estatistica_ecommerce" # Assumindo que o notebook está em notebooks/
charts_dir = "../../charts"
if not os.path.exists(charts_dir): os.makedirs(charts_dir)

# Carregar os dados
df_orders = pd.read_csv(os.path.join(data_dir, "FACT_Orders.csv"))
df_delivery = pd.read_csv(os.path.join(data_dir, "DIM_Delivery.csv"))
df_customer = pd.read_csv(os.path.join(data_dir, "DIM_Customer.csv"))
df_shopping = pd.read_csv(os.path.join(data_dir, "DIM_Shopping.csv"))
df_products = pd.read_csv(os.path.join(data_dir, "DIM_Products.csv"))

## 1. Qualidade e Preparação dos Dados

In [ ]:
# Merge dos DataFrames
df_main = df_orders.merge(df_delivery, on='Id', how='left', suffixes=('_Order', '_Delivery'))
df_main = df_main.merge(df_customer, on='Id', how='left')

# Renomear colunas
df_main.rename(columns={
    'Id': 'Order_ID',
    'payment': 'Payment_Method',
    'P_Sevice': 'Freight_Cost',
    'Status': 'Delivery_Status',
    'Services': 'Delivery_Service',
    'Order_Id': 'Delivery_Order_Id'
}, inplace=True)

# Conversão de tipos (Datas)
date_cols = ['Order_Date', 'D_Date', 'D_Forecast']
for col in date_cols:
    df_main[col] = df_main[col].apply(lambda x: str(x).split('.')[0] if pd.notna(x) else x)
    df_main[col] = pd.to_datetime(df_main[col], errors='coerce')

# Trimming em strings
string_cols = ['Payment_Method', 'Purchase_Status', 'Delivery_Service', 'Delivery_Status', 'UF', 'Region']
for col in string_cols:
    if col in df_main.columns:
        df_main[col] = df_main[col].astype(str).str.strip()

# Tratamento de NAs e Duplicatas
df_main.dropna(subset=['Order_Date'], inplace=True)
df_main.drop_duplicates(subset=['Order_ID'], inplace=True)

# Engenharia de Features (KPIs)
df_main['delivery_delay_days'] = (df_main['D_Date'] - df_main['D_Forecast']).dt.days
df_main['delivery_lead_time'] = (df_main['D_Date'] - df_main['Order_Date']).dt.days
df_main['is_late'] = (df_main['D_Date'] > df_main['D_Forecast']).astype(int)
df_main['is_confirmed'] = (df_main['Purchase_Status'] == "Confirmado").astype(int)
df_main['freight_share'] = df_main['Freight_Cost'] / df_main['Total']
df_main['freight_share'].replace([np.inf, -np.inf], np.nan, inplace=True)
df_main['discount_abs'] = df_main['Discount'] * df_main['Subtotal']
df_main['Order_Month'] = df_main['Order_Date'].dt.month
df_main['Order_Year'] = df_main['Order_Date'].dt.year

# Preparação para Mix (Merge Shopping e Products)
df_mix = df_shopping.merge(df_products, left_on='Product', right_on='Product_Name', how='left', suffixes=('_Item', '_Product'))
df_mix.rename(columns={'Id_Item': 'Shopping_ID', 'Id_Product': 'Product_ID_Dim', 'Price_Item': 'Item_Price_Shopping', 'Price_Product': 'Item_Price_Product'}, inplace=True)
df_mix.rename(columns={'Shopping_ID': 'Order_ID'}, inplace=True)

# Merge final
df_final = df_main.merge(df_mix[['Order_ID', 'Category', 'Subcategory', 'Quantity', 'Item_Price_Product']], on='Order_ID', how='left')

print("Primeiras 5 linhas do DataFrame Final:")
df_final.head()

## 2. Análise Descritiva (EDA)

In [ ]:
# Medidas de Tendência Central e Dispersão
descriptive_stats = df_final[["Total", "delivery_lead_time", "Discount", "Freight_Cost", "delivery_delay_days"]].describe()
print("Estatísticas Descritivas:")
print(descriptive_stats.to_markdown())

In [ ]:
# Histograma e Boxplot para Ticket Médio (Total)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(df_final["Total"], kde=True, bins=50)
plt.title("Distribuição do Ticket Médio (Total)")
plt.xlabel("Valor Total do Pedido (R$)")

plt.subplot(1, 2, 2)
sns.boxplot(y=df_final["Total"])
plt.title("Boxplot do Ticket Médio (Total)")
plt.ylabel("Valor Total do Pedido (R$)")
plt.tight_layout()
plt.savefig(os.path.join(charts_dir, "ticket_distribution.png"))
plt.show()

In [ ]:
# Heatmap de Correlação
corr_cols = df_final[["Total", "Subtotal", "Discount", "Freight_Cost", "delivery_lead_time", "delivery_delay_days", "is_late"]]
correlation_matrix = corr_cols.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Heatmap de Correlação entre Variáveis Numéricas")
plt.savefig(os.path.join(charts_dir, "correlation_heatmap.png"))
plt.show()

In [ ]:
# Série Temporal Mensal da Receita
df_final.set_index("Order_Date", inplace=True)
monthly_revenue = df_final["Total"].resample("M").sum()

plt.figure(figsize=(12, 6))
monthly_revenue.plot(kind="line", marker="o")
plt.title("Receita Mensal (Série Temporal)")
plt.xlabel("Mês")
plt.ylabel("Receita Total (R$)")
plt.grid(True)
plt.savefig(os.path.join(charts_dir, "monthly_revenue.png"))
plt.show()

df_final.reset_index(inplace=True)

## 3. Análise Inferencial (Intervalos de Confiança)

In [ ]:
# IC 95% para Ticket Médio (Total)
ci_total = sms.DescrStatsW(df_final['Total']).tconfint_mean(alpha=0.05)
print("--- Intervalo de Confiança 95% para Ticket Médio (Total) ---")
print(f"Média: R$ {df_final['Total'].mean():.2f}")
print(f"IC 95%: (R$ {ci_total[0]:.2f}, R$ {ci_total[1]:.2f})")

# IC 95% para Proporção de Atraso (is_late)
count_late = df_final['is_late'].sum()
n_late = len(df_final['is_late'])
prop_late = count_late / n_late
ci_prop_late = proportion_confint(count_late, n_late, alpha=0.05, method='normal')
print("\n--- Intervalo de Confiança 95% para Proporção de Atraso ---")
print(f"Proporção de Atraso: {prop_late:.4f} ({prop_late*100:.2f}%)")
print(f"IC 95% (Normal): ({ci_prop_late[0]:.4f}, {ci_prop_late[1]:.4f})")

# IC 95% para Proporção de Cancelamento
count_cancelled = (df_final['Purchase_Status'] == "Cancelado").sum()
n_cancelled = len(df_final['Purchase_Status'])
prop_cancelled = count_cancelled / n_cancelled
ci_prop_cancelled = proportion_confint(count_cancelled, n_cancelled, alpha=0.05, method='normal')
print("\n--- Intervalo de Confiança 95% para Proporção de Cancelamento ---")
print(f"Proporção de Cancelamento: {prop_cancelled:.4f} ({prop_cancelled*100:.2f}%)")
print(f"IC 95% (Normal): ({ci_prop_cancelled[0]:.4f}, {ci_prop_cancelled[1]:.4f})")

## 4. KPIs e Insights

In [ ]:
# Conversão de Pagamento (por payment)
payment_conversion = df_final.groupby('Payment_Method')['is_confirmed'].agg(['sum', 'count'])
payment_conversion['Conversion_Rate'] = payment_conversion['sum'] / payment_conversion['count']
print("Conversão de Pagamento por Método:")
print(payment_conversion.to_markdown())

In [ ]:
# Performance Logística por Services
logistics_performance = df_final.groupby('Delivery_Service').agg(
    Avg_Lead_Time=('delivery_lead_time', 'mean'),
    Late_Rate=('is_late', 'mean'),
    Total_Orders=('Order_ID', 'count')
)
print("Performance Logística por Serviço:")
print(logistics_performance.to_markdown())

In [ ]:
# Mix por Category/Subcategory e elasticidade vs desconto
df_mix_agg = df_final.groupby(['Category', 'Subcategory']).agg(
    Total_Quantity=('Quantity', 'sum'),
    Avg_Discount=('Discount', 'mean'),
    Total_Revenue=('Total', 'sum')
).reset_index()

print("Mix de Produtos (Top 5 por Receita):")
print(df_mix_agg.sort_values(by='Total_Revenue', ascending=False).head(5).to_markdown())

In [ ]:
# Sazonalidade por Região
regional_revenue = df_final.groupby('Region')['Total'].sum().sort_values(ascending=False)
print("Receita por Região:")
print(regional_revenue.to_markdown())